In [7]:
import pandas as pd
import glob
import numpy as np
import plotly.graph_objects as go
from scipy.signal import argrelextrema
import shutil

In [8]:
def candle_color(open, close):
    color = ""
    if(open>close):
        color = "red"
    else:
        color = "green"
    return color

def get_sup_res(data, resistance_ls, support_ls):
    max_idx = argrelextrema(data_new['High'].values, np.greater, order=5)[0]
    min_idx = argrelextrema(data_new['Low'].values, np.less, order=5)[0]
    
    if(len(resistance_ls)==0):
        resistance_ls.append(data.iloc[0]['High'])
    if(len(support_ls)==0):
        support_ls.append(data.iloc[0]['Low'])
        
    if(len(max_idx)>=1):
        for ls in max_idx:
            resistance_ls.append(data.iloc[ls]['High'])
    if(len(min_idx)>=1):
        for ls in min_idx:
            support_ls.append(data.iloc[ls]['Low'])
    
    # Remove duplicate
    resistance_ls = list(set(resistance_ls))
    support_ls = list(set(support_ls))
    
    # Sort
    resistance_ls = sorted(resistance_ls, key=int, reverse=True)
    support_ls = sorted(support_ls, key=int, reverse=False)
    
    return resistance_ls, support_ls
        
    
def get_high_low(data):
    high_ls = data['High'].to_list()
    low_ls = data['Low'].to_list()
    
    
    max_high = max([high_ls[-1], high_ls[-2]])
    min_low = min([low_ls[-1], low_ls[-2]])
    
    return max_high, min_low


In [20]:
## Type 2


data_path = "D:/tradeBot/data/download/2022/04/"

for day in glob.glob(data_path+"*.csv"):
    print(day)
    #day = "D:/tradeBot/data/download/2022/10/2022-10-28.csv"
    data = pd.read_csv(day)
    data = data.dropna()
    count_candle = 0
    resistance_ls = []
    support_ls = []
    is_rg = False
    reserve_candle = []
    is_trade_active = False
    stoploss = 0
    target = 0
    entry = 0
    for _,rows in data.iterrows():


        if(is_rg==False and count_candle>1):
            prev_color = candle_color(data.iloc[count_candle-1]['Open'], data.iloc[count_candle-1]['Close'])
            curr_color = candle_color(rows['Open'], rows['Close'])
            if(prev_color != curr_color):
                is_rg = True
                continue
                
        if(is_rg==True):
            mhigh, mlow = get_high_low(data_new)
            if(rows['Low'] < mlow or rows['High'] > mhigh):
                data_new = data[:count_candle+1]
                res, sup = get_sup_res(data_new, resistance_ls, support_ls)
                if(rows['High'] > mhigh):
                    risk = mhigh - mlow
                    reward = res[0] - mhigh
                    rr = (res[0] - mhigh)/(mhigh - mlow)
                    if(rr > 1.0):
                        is_trade_active = True
                        stoploss = mlow
                        target = res[0]
                        entry = mhigh
                        continue
                if(rows['Low'] < mlow):
                    risk = mhigh - mlow
                    reward = mlow - sup[0]
                    rr = (mlow - sup[0])/(mhigh - mlow)
                    if(rr > 1.0):
                        is_trade_active = True
                        stoploss = mhigh
                        target = sup[0]
                        entry = mlow
                        continue
                        
        if(is_trade_active==True and is_rg==True):
            
            if(rows['High']>=target):
                
                print("################################################################################")
                print(rows['Date'])
                print(rows['Low'],mlow)
                print("rr : "+str(rr))
                data_new = data[:count_candle+2]
                fig_candle = go.Figure(data=go.Candlestick(x=data_new['Date'],
                                     open=data_new['Open'],
                                     high=data_new['High'],
                                     low=data_new['Low'],
                                     close=data_new['Close']))
                fig_candle.add_hline(y=target, line_width=1, line_color="green")
                fig_candle.add_hline(y=stoploss, line_width=1, line_color="red")
                
                fig_candle.add_hline(y=entry, line_width=1, line_color="black")
                #fig_candle.add_hline(y=res[0], line_width=3, line_color="red")

                fig_candle.show()

                print("res : "+str(res))
                print("sup : "+str(sup))
                print("################################################################################")
                #break
        
        count_candle = count_candle + 1
        if(count_candle == 80):
            break

        
    
    break

D:/tradeBot/data/download/2022/04\2022-04-01.csv
################################################################################
2022-04-01 09:19:00+05:30
36403.7 36313.5
rr : 0.0


res : [36388.5, 36342.1]
sup : [36243.25]
################################################################################
################################################################################
2022-04-01 09:20:00+05:30
36440.9 36316.7
rr : -0.3922979263647734


res : [36388.5, 36342.1]
sup : [36243.25]
################################################################################
################################################################################
2022-04-01 09:21:00+05:30
36477.45 36350.25
rr : -0.6922767497988809


res : [36388.5, 36342.1]
sup : [36243.25]
################################################################################
################################################################################
2022-04-01 09:22:00+05:30
36493.05 36403.7
rr : -1.1981747066492336


res : [36388.5, 36342.1]
sup : [36243.25]
################################################################################
################################################################################
2022-04-01 09:23:00+05:30
36492.55 36440.9
rr : -1.698666666666686


res : [36388.5, 36342.1]
sup : [36243.25]
################################################################################
################################################################################
2022-04-01 09:33:00+05:30
36488.95 36477.45
rr : -3.223749999999927


res : [36388.5, 36342.1]
sup : [36243.25]
################################################################################
################################################################################
2022-04-01 09:35:00+05:30
36495.95 36492.55
rr : -4.082962962963049


res : [36388.5, 36342.1]
sup : [36243.25]
################################################################################
################################################################################
2022-04-01 09:36:00+05:30
36499.25 36472.45
rr : 0.0


res : [36526.3, 36388.5, 36342.1]
sup : [36243.25]
################################################################################
################################################################################
2022-04-01 09:37:00+05:30
36501.45 36451.2
rr : 0.6133190118152766


res : [36526.3, 36388.5, 36342.1]
sup : [36243.25]
################################################################################


In [ ]:
# type 1

data_path = "D:/tradeBot/data/download/2022/01/"
type1_path = "D:/algotrade/data/strategy_output/type1/"

for day in glob.glob(data_path+"*.csv"):
    print(day)
    #day = "D:/tradeBot/data/download/2022/10/2022-10-28.csv"
    data = pd.read_csv(day)
    data = data.dropna()
    count_candle = 0
    resistance_ls = []
    support_ls = []
    for _,rows in data.iterrows():
        
            
        if(count_candle == 3):
            open_candle_color = candle_color(data.iloc[0]['Open'], data.iloc[0]['Close'])
            first_candle_color = candle_color(data.iloc[count_candle-2]['Open'], data.iloc[count_candle-2]['Close'])
            second_candle_color = candle_color(data.iloc[count_candle-1]['Open'], data.iloc[count_candle-1]['Close'])
            
            if(open_candle_color == "green" and first_candle_color != second_candle_color):
                data_new = data[:count_candle+1]
                max_high = max(data_new.iloc[0]['High'], data_new.iloc[1]['High'], data_new.iloc[2]['High'])
                if(data_new.iloc[-1]['High'] > max_high):
  
                    res, sup = get_sup_res(data_new, resistance_ls, support_ls)
                    mhigh, mlow = get_high_low(data_new)
                    shutil.copy(day.split(".")[0]+".png", type1_path)

                    print("################################################################################")
                    print(rows['Date'])
                    print("################################################################################")
                    
            if(open_candle_color == "red" and first_candle_color != second_candle_color):
                data_new = data[:count_candle+1]
                min_low = min(data_new.iloc[0]['Low'], data_new.iloc[1]['Low'], data_new.iloc[2]['Low'])
                if(data_new.iloc[-1]['Low'] < min_low):
  
                    res, sup = get_sup_res(data_new, resistance_ls, support_ls)
                    mhigh, mlow = get_high_low(data_new)
                    shutil.copy(day.split(".")[0]+".png", type1_path)

                    print("################################################################################")
                    print(rows['Date'])
                    print("################################################################################")
            
        count_candle = count_candle + 1
    
    #break

In [ ]:
data_path = "D:/tradeBot/data/download/2022/06/"

for day in glob.glob(data_path+"*.csv"):
    print(day)
    day = "D:/tradeBot/data/download/2022/10/2022-10-28.csv"
    data = pd.read_csv(day)
    data = data.dropna()
    count_candle = 0
    resistance_ls = []
    support_ls = []
    for _,rows in data.iterrows():
        if(count_candle == 40):
            break
            
        if(count_candle == 0 or count_candle == 1):
            pass
        else:
            prev_color = candle_color(data.iloc[count_candle-1]['Open'], data.iloc[count_candle-1]['Close'])
            curr_color = candle_color(rows['Open'], rows['Close'])
            if(prev_color != curr_color):
                open_candle_color = candle_color(data.iloc[0]['Open'], data.iloc[0]['Close'])
                if(open_candle_color == "green" and )
                data_new = data[:count_candle+1]
                res, sup = get_sup_res(data_new, resistance_ls, support_ls)
                mhigh, mlow = get_high_low(data_new)

                print("################################################################################")
                print(rows['Date'])
                
                print("count_candle : "+str(count_candle))
                print("prev_color : "+str(prev_color))
                print("curr_color : "+str(curr_color))
                
                data_new = data[:count_candle+2]
                fig_candle = go.Figure(data=go.Candlestick(x=data_new['Date'],
                                     open=data_new['Open'],
                                     high=data_new['High'],
                                     low=data_new['Low'],
                                     close=data_new['Close']))
                fig_candle.add_hline(y=mlow, line_width=3, line_color="green")
                fig_candle.add_hline(y=mhigh, line_width=3, line_color="red")
                
                fig_candle.show()
                
                
                print("res : "+str(res))
                print("sup : "+str(sup))
                print("################################################################################")
                break
            
        count_candle = count_candle + 1
    
    break